#### -----------------------------------------------------------------------------<br>Copyright (c) 2022, Lucid Vision Labs, Inc.
##### THE  SOFTWARE  IS  PROVIDED  "AS IS",  WITHOUT  WARRANTY  OF  ANY  KIND,<br>EXPRESS  OR  IMPLIED,  INCLUDING  BUT  NOT  LIMITED  TO  THE  WARRANTIES<br>OF  MERCHANTABILITY,  FITNESS  FOR  A  PARTICULAR  PURPOSE  AND<br>NONINFRINGEMENT.  IN  NO  EVENT  SHALL  THE  AUTHORS  OR  COPYRIGHT  HOLDERS<br>BE  LIABLE  FOR  ANY  CLAIM,  DAMAGES  OR  OTHER  LIABILITY,  WHETHER  IN  AN<br>ACTION  OF  CONTRACT,  TORT  OR  OTHERWISE,  ARISING  FROM,  OUT  OF  OR  IN<br>CONNECTION  WITH  THE  SOFTWARE  OR  THE  USE  OR  OTHER  DEALINGS  IN  THE  SOFTWARE.<br>-----------------------------------------------------------------------------

In [ ]:
from arena_api.system import system
import time

#### Enumeration MAC: Introduction
>    This example enumerates a device and obtains its MAC address.
    It then verifies it against the MAC address stored in the node map.

In [ ]:
"""
Demonstrates creation of connected devices
"""

tries = 0
tries_max = 6
sleep_time_secs = 10
while tries < tries_max:  # Wait for device for 60 seconds
    devices = system.create_device()
    if not devices:
        print(
            f'Try {tries+1} of {tries_max}: waiting for {sleep_time_secs} '
            f'secs for a device to be connected!')
        for sec_count in range(sleep_time_secs):
            time.sleep(1)
            print(f'{sec_count + 1 } seconds passed ',
                  '.' * sec_count, end='\r')
        tries += 1
    else:
        print(f'Created {len(devices)} device(s)\n')
        break
else:
    raise Exception(f'No device found! Please connect a device and run '
                    f'the example again.')


##### Dynamically get the MAC address of a device that exists so the example passes
> This function can be avoided if
	the user wants to hard code the MAC address instead

In [ ]:
address = ""

device_infos = system.device_infos
if len(device_infos) == 0:
	raise BaseException('No device is found!')
else:
	address = device_infos[0]['mac']
print(f'\'{address}\' is the MAC address to find')

##### Find a specific mac address in device infos then return that device info

In [ ]:
found_device = False

selected_device_info = {}

for device_info in system.device_infos:
	if device_info['mac'] == address:
		found_device = True
		selected_device_info = device_info
		print(f'\'{address}\' device was found')
if not found_device:
	raise BaseException(f'No device with MAC Address '
						f'\'{mac_to_find}\' was found!')

##### Creates selected device

In [ ]:
print('Creating device from device info')
devices_list = system.create_device(device_infos=selected_device_info)
device = devices_list[0]
print(f'Device created: {device}')

##### Simple verification
> Read device's MAC address from the device to verify

In [ ]:
device_mac_int = device.nodemap.get_node('GevMACAddress').value

# Converts int value to MAC address of format XX:XX:XX:XX:XX:XX

hex_mac = f'{device_mac_int:012x}'
device_mac = ':'.join(hex_mac[i:i+2] for i in range(0, len(hex_mac), 2))

# Verify

print(f'Device MAC Address = {device_mac}')
if device_mac != address:
	raise BaseException('The wrong device has been created')

##### Clean up ----------------------------------------------------------------
> Destroy device. This call is optional and will automatically be
  called for any remaining devices when the system module is unloading.

In [ ]:
system.destroy_device()

##### System is closed when module is closed.
>    Explicitly closing it would prevent it from being reopened, 
    as it is only supposed to be opened with "from arena_api.system import system"
